In [1]:
import pandas as pd

In [2]:
#making this notebook to take out regions from possible enhancers
#this is NOT for checking Fst outliers in genes or checking if eg Fst is in UTR

In [3]:
df = pd.read_csv('all_annotations.txt', sep="\t", header=None, names=["chr","source","type","start","stop","s1","s2","s3","info"])

In [4]:
#these are either overlaping or not important due to low sample size
df.drop(df[df['type'] == "region"].index, inplace = True)
df.drop(df[df['type'] == "mRNA"].index, inplace = True)
df.drop(df[df['type'] == "exon"].index, inplace = True)
df.drop(df[df['type'] == "CDS"].index, inplace = True)
df.drop(df[df['type'] == "sequence_feature"].index, inplace = True)
df.drop(df[df['type'] == "tandem_repeat"].index, inplace = True)
df.drop(df[df['type'] == "promoter"].index, inplace = True)
#so we are left with eg gene, lncRNA, rRNA snRNA, etc... we want to take out a region that overlaps with any of these as then they are not cis-regulatory regions

In [5]:
df.type.unique()

array(['gene', 'snRNA', 'pseudogene', 'cDNA_match', 'tRNA', 'lnc_RNA',
       'transcript', 'snoRNA', 'rRNA', 'primary_transcript', 'miRNA'],
      dtype=object)

In [6]:
df=df.drop(columns=['source','type','s1', 's2', 's3','info'])

In [7]:
#df["type"] = "annot"
df=df.sort_values(by=["chr",'start'])
all_chrs=df.chr.unique()
#df["length"] = df["stop"] - df["start"]

#CHECK IF THERE ARE NEGATIVE NUMBERS
#min(df["length"]) <= 0

In [8]:
#check if there is overlap between regions of the same set

In [9]:
#returns TRUE if there is overlap
def check_overlap(a, b):
    return max(0, min(a[1], b[1]) - max(a[0], b[0])) > 0

In [10]:
max_safety=1000000


def get_incorr_set(data,ch):
    tilt=[]
    outdf=pd.DataFrame(columns=["chr", "start","stop"])
    temp_df=data[data["chr"]==ch]
    x=0
    z=0
    while x <= len(temp_df) and z < max_safety:
        state = "go"
        j=0
        y=1
        while state == "go" and j < max_safety and (y+x+1) <= len(temp_df):
            a=temp_df.iloc[x].start, temp_df.iloc[x].stop
            b=temp_df.iloc[x+y].start, temp_df.iloc[x+y].stop
            if check_overlap(a,b) != True:
                state="stop"
                if x not in tilt:
                    outdf=outdf.append({'chr': ch, 'start':a[0], 'stop':a[1]},ignore_index=True)
                    tilt.append(x)
                if x+y not in tilt:
                    outdf=outdf.append({'chr': ch, 'start':b[0], 'stop':b[1]},ignore_index=True)
                    tilt.append(x+y)
            else:
                outdf = outdf.append({'chr': ch, 'start':a[0], 'stop':max(a[1],b[1])},ignore_index=True)
                tilt.append(x)
                tilt.append(x+y)
            y=y+1
            j=j+1
        x=x+1
        z=z+1
    return(outdf)

In [13]:
def get_corr_set(data,ch,debug=False):
    tilt=[]
    outdf=pd.DataFrame(columns=["chr", "start","stop"])
    temp_df=data[data["chr"]==ch]
    x=0
    z=0
    while x <= len(temp_df) and z < max_safety:
        state = "go"
        j=0
        y=1
        while state == "go" and j < max_safety and (y+x+1) <= len(temp_df):
            a=temp_df.iloc[x].start, temp_df.iloc[x].stop
            b=temp_df.iloc[x+y].start, temp_df.iloc[x+y].stop
            if check_overlap(a,b) != True:
                state="stop"
                if x not in tilt:
                    outdf=outdf.append({'chr': ch, 'start':a[0], 'stop':a[1]},ignore_index=True)
                    tilt.append(x)
                if x+y not in tilt:
                    outdf=outdf.append({'chr': ch, 'start':b[0], 'stop':b[1]},ignore_index=True)
                    tilt.append(x+y)
            else:
                outdf = outdf.append({'chr': ch, 'start':a[0], 'stop':max(a[1],b[1])},ignore_index=True)
                if a[1] > b[1]:
                    outdf = outdf[outdf.stop != b[1]]
                if a[1] < b[1]:
                    outdf = outdf[outdf.stop != a[1]]
                else: 
                    print("van baj")
                tilt.append(x)
                tilt.append(x+y)
                if debug:
                    print(a)
                    print(b)
                    print("overlapped")
            y=y+1
            j=j+1
        x=x+1
        z=z+1
    return(outdf)

In [14]:
for ch in all_chrs:
    print(ch)
    fdf= get_incorr_set(df,ch)
    ffdf = get_corr_set(fdf,ch)
    #check if there is still some overlap
    get_corr_set(ffdf,ch,debug=True)
    

NC_001453.1
NW_022144748.1
NW_022144751.1
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(5357, 5549)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlapped
van baj
(4540, 9415)
(4540, 9415)
overlap

KeyboardInterrupt: 

In [16]:
df[df['chr']=="NC_001453.1"].head()

,chr,start,stop
931718,NC_001453.1,1,73
931720,NC_001453.1,74,951
931722,NC_001453.1,939,1009
931724,NC_001453.1,1014,1084
931729,NC_001453.1,1206,1275


LAPO's stuff that is correct (probably)

In [32]:
def get_segs(df, chromosome):
    return [(start,stop) for _,(_,start,stop) in df[df['chr']==chromosome].iterrows()]

In [58]:
src = get_segs(df,"NC_001453.1")
src[:10]

[(1, 73),
 (74, 951),
 (939, 1009),
 (1014, 1084),
 (1206, 1275),
 (1277, 1347),
 (1354, 1425),
 (1426, 1497),
 (1497, 1567),
 (1570, 1639)]

In [71]:
def clean_segments(src):
    if len(src) <= 1:
        return src
    
    a = src[0]
    dest = []
    for b in src[1:]:
        # print(a,b, check_overlap(a,b))
        if check_overlap(a,b):
            a = (min(a[0],b[0]),max(a[1],b[1]))
        else:
            dest.append(a)
            a = b
    dest.append(a)
    return dest

In [72]:
clean_segments(src)

[(1, 73),
 (74, 1009),
 (1014, 1084),
 (1206, 1275),
 (1277, 1347),
 (1354, 1425),
 (1426, 1497),
 (1497, 1567),
 (1570, 1639),
 (1640, 1777),
 (1794, 1867),
 (1868, 1937),
 (1944, 2014),
 (2020, 2087),
 (2088, 2158),
 (2162, 3130),
 (3132, 3200),
 (3201, 4259),
 (4260, 7338),
 (7349, 7415),
 (7418, 7711),
 (7711, 8400),
 (8405, 8475),
 (8476, 9329),
 (9333, 10115),
 (10120, 10184),
 (10201, 10551),
 (10567, 12013),
 (12014, 12081),
 (12082, 14484),
 (14509, 15650)]

In [36]:
next(rest)

(939, 1009)

In [50]:
i = iter([2,3,4])

In [49]:
while True:
    try:
        print(next(i))
    except StopIteration:
        print("Done")
        break

2
3
4
Done


In [51]:
for j in i:
    print(j)

2
3
4
